###Telecom Domain ReadOps & Write Ops Assignment - Building Datalake & Lakehouse

This notebook contains assignments to practice Spark read options and Databricks volumes.  
Sections: Sample data creation, Catalog & Volume creation, Copying data into Volumes, Path glob/recursive reads, toDF() column renaming variants, inferSchema/header/separator experiments, and exercises.

![](https://theciotimes.com/wp-content/uploads/2021/03/TELECOM1.jpg)

###First Import all required libraries & Create spark session object

### 1. Write SQL statements to create:

1. A catalog named telecom_catalog_assign
2. A schema landing_zone
3. A volume landing_vol

In [0]:
%sql
create catalog if not exists telecom_catalog_assign;
create database if not exists telecom_catalog_assign.landing_zone;
create volume if not exists telecom_catalog_assign.landing_zone.landing_vol;

4.Using dbutils.fs.mkdirs, create folders:
/Volumes/telecom_catalog_assign/landing_zone/landing_vol/customer/ /Volumes/telecom_catalog_assign/landing_zone/landing_vol/usage/ /Volumes/telecom_catalog_assign/landing_zone/landing_vol/tower/

In [0]:
dbutils.fs.mkdirs("/Volumes/telecom_catalog_assign/landing_zone/landing_vol/customer")
dbutils.fs.mkdirs("/Volumes/telecom_catalog_assign/landing_zone/landing_vol/usage")
dbutils.fs.mkdirs("/Volumes/telecom_catalog_assign/landing_zone/landing_vol/tower")

5. Explain the difference between (Just google and understand why we are going for volume concept for prod ready systems):  
a. Volume vs DBFS/FileStore  
b. Why production teams prefer Volumes for regulated data

Volumes:
- Managed by Unity Catalog, providing centralized access control, auditing, and organization.
- Primarily intended for non-tabular data (e.g., images, PDFs, libraries, unstructured files).
- Accessed using the secure, structured path /Volumes/<catalog>/<schema>/<volume>/<path>.
- Seamlessly integrates with the Unity Catalog data hierarchy alongside tables and schemas. 

DBFS file storage:
- Uses legacy access patterns (e.g., IAM roles, storage keys configured on the cluster) with limited governance  
- Can store any data
- Accessed using paths like /dbfs/... or legacy mount points (/mnt/...), which are less secure.
- A separate abstraction layer over cloud storage that uses different, less secure access models.

####Why Production teams are prefereing Volumes for regulated data?  
Production teams prefer Volumes for regulated data due to the robust, centralized governance capabilities provided by the Unity Catalog, which are essential for meeting stringent security and compliance standards like HIPAA and PCI-DSS  
Key reasons for this preference include:  
**Centralized and Fine-Grained Access Control**: Volumes allow administrators to define access policies using standard ANSI SQL at a granular level (catalog, schema, volume, or even file level), ensuring that only authorized users or groups can access sensitive non-tabular data. In contrast, DBFS mounts often provide broad access across the workspace, which is a security concern for regulated data.  
**Comprehensive Auditing and Lineage**: Unity Catalog automatically captures user-level audit logs and end-to-end data lineage, which is crucial for compliance verification and audit readiness. This provides transparency into how data moves and is consumed, a feature largely absent in the legacy DBFS.  
**Compliance-by-Design Environment**: Databricks has specific features, like the Compliance Security Profile, designed to help organizations meet industry standards such as HIPAA, PCI-DSS, and FedRAMP. Using volumes within this framework helps enforce enhanced security settings across the storage layer.  
**Abstraction of Cloud Credentials**: With managed volumes, production teams don't need to manually manage complex cloud credentials or storage paths. This simplifies data management and reduces the risk of credential exposure. 
**Isolation and Secure Sharing**: Volumes allow for better data isolation. Catalogs can be bound to specific workspaces, ensuring that production data is processed only in designated, secure environments, and sensitive data can be securely shared with external partners using Delta Sharing without data duplication.  
**Deprecation of DBFS**: Databricks strongly recommends against using the DBFS root and mounts for most use cases in Unity Catalog-enabled workspaces because they lack these critical governance features, making volumes the go-to solution for production workloads.


###Data files to use in this usecase:


### 2. Filesystem operations
1. Write code to copy the above datasets into your created Volume folders: Customer → /Volumes/.../customer/ Usage → /Volumes/.../usage/ Tower (region-based) → /Volumes/.../tower/region1/ and /Volumes/.../tower/region2/
2. Write a command to validate whether files were successfully copied

In [0]:
#customer_csv = ''' 101,Arun,31,Chennai,PREPAID 102,Meera,45,Bangalore,POSTPAID 103,Irfan,29,Hyderabad,PREPAID 104,Raj,52,Mumbai,POSTPAID 105,,27,Delhi,PREPAID 106,Sneha,abc,Pune,PREPAID '''

customer_path = "/Volumes/telecom_catalog_assign/landing_zone/landing_vol/customer/customer_csv.txt"

dbutils.fs.put("/Volumes/telecom_catalog_assign/landing_zone/landing_vol/customer/customer_csv",customer_path, overwrite=True)

In [0]:
print(dbutils.fs.head("/Volumes/telecom_catalog_assign/landing_zone/landing_vol/customer/customer_csv.txt"))

In [0]:
#usage_tsv = '''customer_id\tvoice_mins\tdata_mb\tsms_count\n101\t320\t1500\t20\n102\t120\t4000\t5\n103\t540\t600\t52\n104\t45\t200\t2\n105\t0\t0\t0 '''
usage_tsv = "/Volumes/telecom_catalog_assign/landing_zone/landing_vol/usage/usage_tsv.txt"

volume_path = "/Volumes/telecom_catalog_assign/landing_zone/landing_vol/usage/usage_csv"

dbutils.fs.put(volume_path, usage_tsv,overwrite=True)
print(dbutils.fs.head(volume_path))

In [0]:
tower_logs_region1 = '''event_id|customer_id|tower_id|signal_strength|timestamp 5001|101|TWR01|-80|2025-01-10 10:21:54 5004|104|TWR05|-75|2025-01-10 11:01:12 '''

volume_path = "/Volumes/telecom_catalog_assign/landing_zone/landing_vol/tower/tower_logs_region1"

dbutils.fs.put(volume_path, tower_logs_region1,overwrite=True)
print(dbutils.fs.head(volume_path))

In [0]:
dbutils.fs.cp("/Volumes/telecom_catalog_assign/landing_zone/landing_vol/tower/tower_logs_region1","/Volumes/telecom_catalog_assign/landing_zone/landing_vol/tower/tower_logs_region2")

In [0]:
dbutils.fs.cp("/Volumes/telecom_catalog_assign/landing_zone/landing_vol/tower/tower_logs_region1","/Volumes/telecom_catalog_assign/landing_zone/landing_vol/tower/tower_logs_region3.csv")

In [0]:
dbutils.fs.cp("/Volumes/telecom_catalog_assign/landing_zone/landing_vol/customer/customer_csv","/Volumes/telecom_catalog_assign/landing_zone/landing_vol/customer/customer.csv")

In [0]:
dbutils.fs.cp("/Volumes/telecom_catalog_assign/landing_zone/landing_vol/usage/usage_tsv","/Volumes/telecom_catalog_assign/landing_zone/landing_vol/usage/usage.csv")

### 3. Directory Read Use Cases
1. Read all tower logs using: Path glob filter (example: *.csv) Multiple paths input Recursive lookup

2. Demonstrate these 3 reads separately: Using pathGlobFilter Using list of paths in spark.read.csv([path1, path2]) Using .option("recursiveFileLookup","true")

3. Compare the outputs and understand when each should be used.

In [0]:
file_path = "/Volumes/telecom_catalog_assign/landing_zone/landing_vol/tower/"

#Read the csv files using pathGlobFilter
tower_df = spark.read.format("csv").option("header","true").option("pathGlobFilter","*.csv").load(file_path)

display(tower_df.head)

In [0]:
file_path = "/Volumes/telecom_catalog_assign/landing_zone/landing_vol/"

recursive_df = spark.read.format("csv").option("header","true").option("recursiveFileLookup","true").load(file_path)

display(recursive_df)

### 4. Schema Inference, Header, and Separator

1. Try the Customer, Usage files with the option and options using read.csv and format function:
header=false, inferSchema=false
or
header=true, inferSchema=true

In [0]:
df1 = spark.read.csv("/Volumes/telecom_catalog_assign/landing_zone/landing_vol/customer/customer_csv.txt", header=True, inferSchema=False)

df1.show()

In [0]:
df2 = spark.read.csv("/Volumes/telecom_catalog_assign/landing_zone/landing_vol/customer/customer_csv.txt", header=False, inferSchema=True)

df2.show()

###5. Column Renaming Usecases

In [0]:
# 1. Apply column names using string using toDF function for customer data
df3 = spark.read.csv("/Volumes/telecom_catalog_assign/landing_zone/landing_vol/customer/customer_csv.txt").toDF("id","name","age","location","plan")

df3.show(2)

In [0]:
#2. Apply column names and datatype using the schema function for usage data
df4 = spark.read.csv("/Volumes/telecom_catalog_assign/landing_zone/landing_vol/usage/usage_tsv.txt",sep="\t",header=True,inferSchema=False).toDF("cust_id","voice_mins","data_mb","sms_count")

df3.show()

In [0]:
# 3. Apply column names and datatype using the StructType with IntegerType, StringType, TimestampType and other classes for towers data
from pyspark.sql.types import StructType, StructField, IntegerType, StringType, TimestampType

custom_schema = StructType([
    StructField("event_id", StringType(), True),
    StructField("customer_id", IntegerType(), True),
    StructField("tower_id", StringType(), True),
    StructField("signal_strength", StringType(), True),
    StructField("timestamp", TimestampType(), True)
])

tower_df = spark.read.csv("/Volumes/telecom_catalog_assign/landing_zone/landing_vol/tower/tower_logs_region3.csv", schema=custom_schema)

tower_df.show(5)

###Spark Write Operations using
- csv, json, orc, parquet, delta, saveAsTable, insertInto, xml with different write mode, header and sep options

###6. Write Operations (Data Conversion/Schema migration) – CSV Format Usecases

1. Write customer data into CSV format using overwrite mode

In [0]:
df3.write.csv(path="/Volumes/telecom_catalog_assign/landing_zone/landing_vol/customer/csvout",header=True,sep=",",mode="overwrite")

2. Write usage data into CSV format using append mode

In [0]:
df4.write.csv(path="/Volumes/telecom_catalog_assign/landing_zone/landing_vol/usage/usgout",header=True,sep=",",mode="overwrite")

3. Write tower data into CSV format with header enabled and custom separator (|)

In [0]:
#need to workout tower data

4. Read the tower data in a dataframe and show only 5 rows.

5. Download the file into local from the catalog volume location and see the data of any of the above files opening in a notepad++.

In [0]:
#working fine

### 7. Write Operations (Data Conversion/Schema migration)– JSON Format Usecases

1. Write customer data into JSON format using overwrite mode

In [0]:
df3.write.json(path="/Volumes/telecom_catalog_assign/landing_zone/landing_vol/customer/jsonout",mode="overwrite")

2. Write usage data into JSON format using append mode and snappy compression format

In [0]:
df4.write.json(path="/Volumes/telecom_catalog_assign/landing_zone/landing_vol/usage/jsonout",mode="append",compression="snappy")

3. Write tower data into JSON format using ignore mode and observe the behavior of this mode

In [0]:
#need to work out

4. Read the tower data in a dataframe and show only 5 rows.

5. Download the file into local harddisk from the catalog volume location and see the data of any of the above files opening in a notepad++.

In [0]:
#working fine

###8. Write Operations (Data Conversion/Schema migration) – Parquet Format Usecases

1. Write customer data into Parquet format using overwrite mode and in a gzip format

In [0]:
df3.write.parquet(path="/Volumes/telecom_catalog_assign/landing_zone/landing_vol/customer/parquetout",mode="overwrite",compression="gzip")

2. Write usage data into Parquet format using error mode

In [0]:
df4.write.parquet(path="/Volumes/telecom_catalog_assign/landing_zone/landing_vol/usage/parquetout",mode="error")

3. Write tower data into Parquet format with gzip compression option

In [0]:
#Need to work

4. Read the usage data in a dataframe and show only 5 rows.

In [0]:
spark.read.parquet("/Volumes/telecom_catalog_assign/landing_zone/landing_vol/usage/parquetout").show(5)

5. Download the file into local harddisk from the catalog volume location and see the data of any of the above files opening in a notepad++

In [0]:
#working fine

###9. Write Operations (Data Conversion/Schema migration) – Orc Format Usecases

1. Write customer data into ORC format using overwrite mode

In [0]:
df3.write.orc(path="/Volumes/telecom_catalog_assign/landing_zone/landing_vol/customer/orcout",mode="overwrite",compression="zlib")

2. Write usage data into ORC format using append mode

In [0]:
df4.write.orc(path="/Volumes/telecom_catalog_assign/landing_zone/landing_vol/usage/orcout",mode="append")

3. Write tower data into ORC format and see the output file structure

In [0]:
#Need to work

4. Read the usage data in a dataframe and show only 5 rows.

In [0]:
usage_orc_df = spark.read.orc("/Volumes/telecom_catalog_assign/landing_zone/landing_vol/usage/orcout")
usage_orc_df.show()

5. Download the file into local harddisk from the catalog volume location and see the data of any of the above files opening in a notepad++.

In [0]:
#working fine

###10. Write Operations (Data Conversion/Schema migration) – Delta Format Usecases

1. Write customer data into Delta format using overwrite mode

In [0]:
df3.write.format("delta").save("/Volumes/telecom_catalog_assign/landing_zone/landing_vol/customer/deltaout",mode="overwrite")

2. Write usage data into Delta format using append mode

In [0]:
df4.write.format("delta").save("/Volumes/telecom_catalog_assign/landing_zone/landing_vol/usage/deltaout",mode="append")

3. Write tower data into Delta format and see the output file structure

In [0]:
#Need to check

4. Read the usage data in a dataframe and show only 5 rows.

In [0]:
usage_deltadf = spark.read.format("delta").load("/Volumes/telecom_catalog_assign/landing_zone/landing_vol/usage/deltaout")
usage_deltadf.show(5)

5. Download the file into local harddisk from the catalog volume location and see the data of any of the above files opening in a notepad++

In [0]:
#working fine

6. Compare the parquet location and delta location and try to understand what is the differentiating factor, as both are parquet files only.

In [0]:
#Parquet location is having started, commited and part file, whereas Delta location is having only Part file along with delta_log file. _delta_log file contains the metadata and _staged_commits. Whereas Parquet is having success and commited files

###11. Write Operations (Lakehouse Usecases) – Delta table Usecases

1. Write customer data using saveAsTable() as a managed table

In [0]:
df3.write.saveAsTable("workspace.wd36schema.lh_custtbl", mode="overwrite")

2. Write usage data using saveAsTable() with overwrite mode

In [0]:
df4.write.saveAsTable("workspace.wd36schema.lh_usgtbl", mode="overwrite")

3. Drop the managed table and verify data removal

In [0]:
%sql
DROP TABLE IF EXISTS workspace.wd36schema.lh_custtbl;

4. Go and check the table overview and realize it is in delta format in the Catalog.

Not sure what the question is

5. Use spark.read.sql to write some simple queries on the above tables created.

In [0]:
display(spark.sql("select * from workspace.wd36schema.lh_usgtbl"))

###12. Write Operations (Lakehouse Usecases) – Delta table Usecases

1. Write customer data using insertInto() in a new table and find the behavior

In [0]:
df3.write.insertInto("workspace.wd36schema.lh_custtbl", overwrite=True)

In [0]:
# Above statement shows insertinto() needs table to be created prior to executing the statement.New table cannot be created by default.

2. Write usage data using insertTable() with overwrite mode

In [0]:
df4.write.insertInto("workspace.wd36schema.lh_usgtbl",overwrite=True)

### 13. Write Operations (Lakehouse Usecases) – Delta table Usecases

1. Write customer data into XML format using rowTag as cust

In [0]:
df3.write.xml("/Volumes/telecom_catalog_assign/landing_zone/landing_vol/customer/xmlout",mode='overwrite',rowTag="cust")

2. Write usage data into XML format using overwrite mode with the rowTag as usage

In [0]:
df4.write.xml("/Volumes/telecom_catalog_assign/landing_zone/landing_vol/usage/xmlout",rowTag="usage",mode="overwrite")

3. Download the xml data and open the file in notepad++ and see how the xml file looks like.

### 14. Compare all the downloaded files (csv, json, orc, parquet, delta and xml)


1.Capture the size occupied between all of these file formats and list the formats below based on the order of size from small to big.

In [0]:
"""
csv = 106 bytes
json = 344 bytes
ORC = 685 bytes
parquet = 1.25 kb
delta = 1.25 kb
xml = 874 bytes

Ordered based on size
csv < json < orc < xml < parquet < delta
"""

### 15. Try to do permutation and combination of performing Schema Migration & Data Conversion operations like...

1.Read any one of the above orc data in a dataframe and write it to dbfs in a parquet format

In [0]:
orcdf = spark.read.orc("/Volumes/telecom_catalog_assign/landing_zone/landing_vol/customer/orcout/part-00000-tid-75013927119242283-7cf32933-e9e9-47f7-bc03-f603dac0dfcc-218-1-c000.zlib.orc")
orcdf.write.parquet("/Volumes/telecom_catalog_assign/landing_zone/landing_vol/customer/parquettestout",mode="overwrite")

2. Read any one of the above parquet data in a dataframe and write it to dbfs in a delta format

In [0]:
pardf = spark.read.parquet("/Volumes/telecom_catalog_assign/landing_zone/landing_vol/customer/parquetout/part-00000-tid-4123262550380396419-e63f8747-44d4-4ef4-8940-b2751ef98d11-286-1-c000.gz.parquet")
pardf.write.format("delta").save("/Volumes/telecom_catalog_assign/landing_zone/landing_vol/customer/deltatestout",mode="overwrite")

3. Read any one of the above delta data in a dataframe and write it to dbfs in a xml format

In [0]:
deltadf = spark.read.format("delta").load("/Volumes/telecom_catalog_assign/landing_zone/landing_vol/customer/deltatestout")
deltadf.write.xml("/Volumes/telecom_catalog_assign/landing_zone/landing_vol/customer/xmltestout",rowTag="cust_id",mode="overwrite")

- when we want to read a single Delta file using the Delta format, but Delta expects a directory containing the Delta transaction log and data files, not an individual Parquet file. To fix this, load the entire Delta table directory instead of a single file.
- when we write in xml format rowTag is mandatory

4. Read any one of the above delta table in a dataframe and write it to dbfs in a json format

In [0]:
deltadf.write.json("/Volumes/telecom_catalog_assign/landing_zone/landing_vol/customer/jsontestout",mode="overwrite")

5. Read any one of the above delta table in a dataframe and write it to another table

In [0]:
deltadf.write.saveAsTable('workspace.wd36schema.lh_newtbl')

#### 16. Do a final exercise of defining one/two liner of...

1. When to use/benifits csv
2. When to use/benifits json
3. When to use/benifit orc
4. When to use/benifit parquet
5. When to use/benifit delta
6. When to use/benifit xml
7. When to use/benifit delta tables